In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
DATA_DIR = "drive/My Drive/Colab Notebooks/birds/data"

In [0]:
import os

In [0]:
os.listdir(DATA_DIR)

['Zaunkönig',
 'Zwergschnäpper',
 'Wanderfalke',
 'Sumpfrohrsänger',
 'Haubenmeise',
 'Haussperling Spatz',
 'Grünfink Grünling',
 'Heckenbraunelle',
 'Sperber',
 'Goldammer',
 'Gimpel Dompfaff',
 'Girlitz',
 'Dohle',
 'Gartenrotschwanz',
 'Neuntöter',
 'Ringeltaube',
 'Teichhuhn',
 'Schilfrohrsänger',
 'Haubenlerche',
 'Buchfink',
 'Trauerschnäpper',
 'Hausrotschwanz',
 'Gartenbaumläufer',
 'Sumpfmeise',
 'Eisvogel',
 'Wintergoldhähnchen',
 'Zwergtaucher',
 'Waldkauz',
 'Grauspecht',
 'Reiherente',
 'Rauchschwalbe',
 'Tüpfelsumpfhuhn',
 'Singdrossel',
 'Mehlschwalbe',
 'Höckerschwan',
 'Saatkrähe',
 'Baumfalke',
 'Stockente',
 'Kleiber',
 'Teichrohrsänger',
 'Birkenzeisig',
 'Buntspecht',
 'Rabenkrähe',
 'Zeisig Erlenzeisig',
 'Mittelspecht',
 'Graureiher',
 'Blässhuhn',
 'Kohlmeise',
 'Kleinspecht',
 'Schwarzspecht',
 'Hohltaube',
 'Drosselrohrsänger',
 'Rotkehlchen',
 'Grünspecht',
 'Mönchsgrasmücke',
 'Eichelhäher',
 'Kleines Sumpfhuhn',
 'Elster',
 'Zilpzalp',

In [0]:
import keras
from keras import backend as K
from keras.layers.core import Dense, Activation
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image
from keras.optimizers import Adam

Using TensorFlow backend.


In [0]:
mobile = keras.applications.mobilenet.MobileNet()

In [0]:
def prepare_image(file):
    img_path = ''
    img = image.load_img(img_path + file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array_expanded_dims = np.expand_dims(img_array, axis=0)
    return keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)

In [0]:
base_model=MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=Dense(1024,activation='relu')(x) #dense layer 2
x=Dense(512,activation='relu')(x) #dense layer 3
preds=Dense(84,activation='softmax')(x) #final layer with softmax activation

/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


17227776/17225924 [==============================] - 12s 1us/step


In [0]:
model=Model(inputs=base_model.input,outputs=preds)
#specify the inputs
#specify the outputs
#now a model has been created based on our architecture

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128       
__________

In [0]:
for layer in model.layers:
    layer.trainable=False
# or if we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

In [0]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

In [0]:
train_generator=train_datagen.flow_from_directory(DATA_DIR,
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 7730 images belonging to 84 classes.


In [0]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=15)

Instructions for updating:
Use tf.cast instead.
Epoch 1/15
 41/241 [====>.........................] - ETA: 33:20 - loss: 4.1552 - acc: 0.0640

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:709: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


140/241 [================>.............] - ETA: 16:52 - loss: 3.5067 - acc: 0.1475

/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 4718592 bytes but only got 0. Skipping tag 0
  "Skipping tag %s" % (size, len(data), tag))
/usr/local/lib/python3.6/dist-packages/PIL/TiffImagePlugin.py:692: UserWarning: Possibly corrupt EXIF data.  Expecting to read 2284257280 bytes but only got 0. Skipping tag 0
  "Skipping tag %s" % (size, len(data), tag))


241/241 [==============================] - 2395s 10s/step - loss: 3.1499 - acc: 0.1981
Epoch 2/15
241/241 [==============================] - 225s 933ms/step - loss: 2.1034 - acc: 0.4108
Epoch 3/15
241/241 [==============================] - 221s 915ms/step - loss: 1.6341 - acc: 0.5265
Epoch 4/15
241/241 [==============================] - 220s 914ms/step - loss: 1.3473 - acc: 0.6026
Epoch 5/15
241/241 [==============================] - 220s 911ms/step - loss: 1.0788 - acc: 0.6824
Epoch 6/15
241/241 [==============================] - 222s 919ms/step - loss: 0.9548 - acc: 0.7233
Epoch 7/15
241/241 [==============================] - 219s 910ms/step - loss: 0.8189 - acc: 0.7573
Epoch 8/15
241/241 [==============================] - 218s 906ms/step - loss: 0.6950 - acc: 0.7921
Epoch 9/15
241/241 [==============================] - 220s 913ms/step - loss: 0.6472 - acc: 0.8043
Epoch 10/15
241/241 [==============================] - 223s 925ms/step - loss: 0.5884 - acc: 0.8251
Epoch 11/15
241/241 [

In [0]:
model.save_weights("drive/My Drive/Colab Notebooks/birds/mobilenet.h5")

In [0]:
def load_image(img_path, show=False):

    img = image.load_img(img_path, target_size=(150, 150))
    img_tensor = image.img_to_array(img)                    # (height, width, channels)
    img_tensor = np.expand_dims(img_tensor, axis=0)         # (1, height, width, channels), add a dimension because the model expects this shape: (batch_size, height, width, channels)
    img_tensor /= 255.                                      # imshow expects values in the range [0, 1]

    if show:
        plt.imshow(img_tensor[0])                           
        plt.axis('off')
        plt.show()

    return img_tensor

  




In [0]:
img_path = 'drive/My Drive/Colab Notebooks/birds/test/Schwanzmeise/schwanzmeise_1204610_57923342.jpg'
new_image = prepare_image(img_path)



In [0]:
y_prob = model.predict(new_image)
y_classes = y_prob.argmax(axis=-1)

In [80]:
y_prob

array([[1.87776692e-03, 8.68186122e-04, 2.07542635e-05, 2.74556398e-04,
        3.18712384e-01, 1.22291315e-03, 1.56753689e-01, 2.46160082e-04,
        6.17516343e-04, 4.03260310e-05, 1.05382560e-03, 1.60973432e-04,
        2.96752015e-03, 3.49598820e-04, 2.89365962e-05, 2.09866744e-03,
        1.07855812e-01, 4.41219250e-04, 6.92691712e-04, 9.08845738e-02,
        6.89323933e-05, 3.61528146e-05, 7.13358750e-05, 1.95674875e-04,
        1.80324074e-03, 5.38304634e-03, 3.21668951e-04, 1.80464103e-05,
        1.00877825e-02, 4.50985826e-04, 2.95904689e-02, 1.08625926e-03,
        3.23992153e-03, 3.68670764e-04, 1.66961792e-04, 3.68564646e-03,
        6.10370596e-04, 1.03670813e-03, 2.31657759e-03, 6.57511083e-03,
        3.19114450e-04, 2.29063649e-02, 1.60033902e-04, 1.90455994e-05,
        3.57925193e-04, 3.03497706e-02, 3.89704742e-02, 8.69863085e-04,
        1.74559199e-03, 6.69944930e-05, 1.59774115e-03, 3.79160338e-04,
        2.51871091e-03, 2.91734643e-04, 1.49517285e-03, 9.898332

In [0]:
predicted_label = sorted(labels)[y_classes[0]]

In [82]:
new_image

array([[[[ 0.6862745 ,  0.77254903,  0.2313726 ],
         [ 0.67058825,  0.77254903,  0.15294123],
         [ 0.6627451 ,  0.75686276,  0.16078436],
         ...,
         [ 0.41960788,  0.58431375,  0.12156868],
         [ 0.45882356,  0.6313726 ,  0.12941182],
         [ 0.5058824 ,  0.6627451 ,  0.20000005]],

        [[ 0.69411767,  0.78039217,  0.26274514],
         [ 0.6784314 ,  0.77254903,  0.20784318],
         [ 0.6627451 ,  0.75686276,  0.16078436],
         ...,
         [ 0.48235297,  0.64705884,  0.20000005],
         [ 0.45882356,  0.6313726 ,  0.14509809],
         [ 0.4901961 ,  0.654902  ,  0.19215691]],

        [[ 0.69411767,  0.78039217,  0.26274514],
         [ 0.6784314 ,  0.77254903,  0.19215691],
         [ 0.654902  ,  0.75686276,  0.13725495],
         ...,
         [ 0.427451  ,  0.58431375,  0.10588241],
         [ 0.45098042,  0.60784316,  0.11372554],
         [ 0.4901961 ,  0.6627451 ,  0.17647064]],

        ...,

        [[ 0.27843142,  0.5058824 , -0

In [83]:
predicted_label

'Blaumeise'